Import required elements

In [1]:
from src.keras_model.lenet5_keras_model import LeNet5
from src.datasets.dataset_generators import DatasetGenerators
from keras.losses import categorical_crossentropy

Using TensorFlow backend.


Setup basic params

In [2]:
dataset_dir = '/Users/igor/Python/CNN-Architecture-Comparison/data'
dataset_name = 'mnist'
dataset_image_size = (32, 32)
dataset_grayscale = True
model_optimizer = 'adam'
model_loss = categorical_crossentropy
model_metrics = ['accuracy']
fit_epochs = 10
evaluation_bath_size = 128

Load dataset (MNIST)

In [3]:
dataset_generators = DatasetGenerators(
    name=dataset_name,
    image_size=dataset_image_size,
    dataset_dir=dataset_dir,
    grayscale=dataset_grayscale
)

Loading train set for mnist...
Found 48291 images belonging to 10 classes.
Loading test set for mnist...
Found 13799 images belonging to 10 classes.
Loading validation set for mnist...
Found 6902 images belonging to 10 classes.


Create & compile model

In [4]:
model = LeNet5(
    classes=dataset_generators.test.num_classes
).getModel()

model.compile(
    optimizer=model_optimizer,
    loss=model_loss,
    metrics=model_metrics
)

Perform epoch learning

In [5]:
learn_history = model.fit_generator(
    generator=dataset_generators.train,
    epochs=fit_epochs,
    validation_data=dataset_generators.test
)

Epoch 1/10
1510/1510 [==============================] - 16s 10ms/step - loss: 0.3648 - accuracy: 0.9279 - val_loss: 0.0651 - val_accuracy: 0.9635

Evaluate model

In [6]:
model.evaluate_generator(
    generator=dataset_generators.validate
)


[0.0061775026842951775, 0.9872500896453857]